In [1]:
from iuvs import io
import pandas as pd
l1bfiles = io.l1b_filenames()

In [2]:
from IPython.parallel import Client
rc = Client()

In [3]:
dview = rc.direct_view()
lview = rc.load_balanced_view()

In [34]:
def get_header_df(header, drop_comment=True):
    import pandas as pd
    df = pd.DataFrame(header.values(),
                      index=header.keys())
    return df.drop('COMMENT KERNEL'.split()) if drop_comment else df

def save_to_hdf(df, basename):
    import os
    HOME = os.environ['HOME']
    newfname = os.path.splitext(basename)[0] + '.h5'
    path = os.path.join(HOME, 'output', newfname)
    print(path)
    df.to_hdf(path, 'df', format='t')
    
def process_img_headers(fname):
    from iuvs import io
    import os
    from astropy.io import fits
    iuvsfname = io.Filename(fname)
    l1b = io.L1BReader(fname)
    hdu = l1b.hdulist[0]
    hdu.verify('fix')
    df = get_header_df(hdu.header).T.convert_objects()
    basename = os.path.basename(fname)
    df['filename'] = basename
    save_to_hdf(df, basename)
    return

In [31]:
dview.push({'get_header_df':get_header_df,
            'save_to_hdf': save_to_hdf})

<AsyncResult: finished>

In [35]:
results = lview.map_async(process_img_headers, l1bfiles)

In [ ]:
import time
while not results.ready():
    print("{:.1f} %".format(100*results.progress/len(l1bfiles)))
    time.sleep(30)          

2.1 %
2.3 %
2.7 %

In [23]:
l1b = process_img_headers(l1bfiles[0])

In [24]:
hdu = l1b.hdulist[0]
hdu.verify('silentfix')

In [25]:
hdu.header

SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                  -32 /                                                
NAXIS   =                    2 /                                                
NAXIS1  =                  512 /                                                
NAXIS2  =                  512 /                                                
EXTEND  =                    T /Extensions may be present                       
COMMENT MAVEN IUVS Level 1B Data Product                                        
COMMENT IUVS Principal Investigator N. Schneider                                
COMMENT Laboratory for Atmospheric and Space Physics                            
COMMENT University of Colorado Boulder                                          
COMMENT 1234 Innovation Drive, Boulder CO 80303                                 
COMMENT MAVEN Mission scientific and model results are open to all.             
COMMENT Users should contact